In [1]:
! pip install -U spacy -q 

     |████████████████████████████████| 6.0 MB 5.5 MB/s 
     |████████████████████████████████| 451 kB 70.7 MB/s 
     |████████████████████████████████| 10.1 MB 49.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 181 kB 57.0 MB/s 
     |████████████████████████████████| 628 kB 60.3 MB/s 


In [4]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.6.1                         
Location         c:\Users\VishalMSingh\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy
Platform         Windows-10-10.0.19045-SP0     
Python version   3.10.11                       
Pipelines        en_core_web_sm (3.6.0)        



In [5]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

--2021-12-30 17:13:03--  https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829 (2.8K) [text/plain]
Saving to: ‘pvr_training_data.json’

pvr_training_data.j 100%[===================>]   2.76K  --.-KB/s    in 0s      

2021-12-30 17:13:03 (58.1 MB/s) - ‘pvr_training_data.json’ saved [2829/2829]



In [5]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [6]:
import json
f = open('train21.json')
TRAIN_DATA = json.load(f)

In [7]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./train21.spacy") # save the docbin object

100%|██████████| 34/34 [00:00<00:00, 2615.62it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [8]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [9]:
! python -m spacy train config.cfg --output ./ --paths.train ./train21.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     63.97    0.00    0.00    0.00    0.00
 67     200        294.29   2879.15   27.03   20.00   41.67    0.27
164     400        118.68     60.00   36.36   28.57   50.00    0.36
264     600          0.00      0.00   36.36   28.57   50.00    0.36
364     800          0.00      0.00   37.50   30.00   50.00    0.37
563    1000          0.00      0.00   37.50   30.00   50.00    0.37
763    1200          0.00      0.00   37.50   30.00   50.00    0.37
963    1400          0.00      0.00   37.50   30.00   50.00    0.37
1163    1600          0.00      0.00

In [3]:
nlp_ner = spacy.load("model-best") 

NameError: name 'spacy' is not defined

In [2]:
doc = nlp_ner('''
Morgan Stanley Advantage Services Pvt. Ltd
. Athena Bldg No.5, Sector 30 Mindspace, 
              Goregaon (W) Mumbai PAYSLIP FOR THE MONTH OF NOVEMBER 2020 Emp Code 95RWK Emp Name Monsoor Choudhury Department TAX BLR Cost Center 4P07 PF No. MHBAN0046775000101 3953 Location Bengaluru Designation Sr Associate ESI No. Date of Birth 11-09-1991 Bank A/c No 50100300732580 Pan No. AQFPC0159B Date of Joining 22-07-2019 Gender M EPS No. UAN 100717017168 Regime Type Old Regime Earnings Amount YTD Deductions Amount YTD Basic 37,542.00 300,336.00 Provident Fund 4,505.00 33,787.00 Conveyance Allowance 1,600.00 12,800.00 Profession Tax 200.00 1,600.00 Other Earnings 2,000.00 12,000.00 Medical Insurance 1,399.00 11,192.00 House Rent Allowance 35,942.00 287,536.00 Income Tax 3,141.00 34,273.00 Total Earnings 77,084.00 612,672.00 Total Deductions 9,245.00 80,852.00 Net Pay : Rs. 67,839.00 In Words Rupees Sixty Seven Thousand Eight Hundred and Thirty Nine Only. Private and Confidential.This is a system-generated document and signature is not required'
''') # input sample text

NameError: name 'nlp_ner' is not defined

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter